In [1]:
from __future__ import print_function, division
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import librosa
import wave, os, glob

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="6"

# Loading the Data

I have made use of the glob utility in python to load the data faster. The signals are stored in a list after doing STFT.

In [ ]:
import glob
noisy_list = []; clean_list = []; noise_list = []; complex_noisy = []
for filename in sorted(glob.iglob('/opt/e533/timit-homework/tr/trx*.wav')):
    noisy_data, sr=librosa.load(filename, sr=None)
    X=librosa.stft(noisy_data, n_fft=1024, hop_length=512)
    noisy_list.append(np.transpose(np.abs(X)))
    complex_noisy.append(np.transpose(X))

for filename in sorted(glob.iglob('/opt/e533/timit-homework/tr/trs*.wav')):
    clean_data, sr=librosa.load(filename, sr=None)
    S=librosa.stft(clean_data, n_fft=1024, hop_length=512)
    clean_list.append(np.transpose(np.abs(S)))

for filename in sorted(glob.iglob('/opt/e533/timit-homework/tr/trn*.wav')):
    noise_only, sr=librosa.load(filename, sr=None)
    N=librosa.stft(noise_only, n_fft=1024, hop_length=512)
    noise_list.append(np.transpose(np.abs(N)))

In [ ]:
test_data = []
complex_test = []

import os, re
for f in sorted(glob.iglob("/opt/e533/timit-homework/te/te*")):
        st, sr=librosa.load(f, sr=None)
        X=librosa.stft(st, n_fft=1024, hop_length=512)
        X_abs = np.abs(X)
        test_data.append(np.transpose(X_abs))
        complex_test.append(np.transpose(X))

In [2]:
for filename in sorted(glob.iglob('/opt/e533/timit-homework/tr/trx0000.wav')):
    noisy_data_f, sr=librosa.load(filename, sr=None)

Dumping and loading the data from pickle files.

In [3]:
import pickle
with open('train.pkl','rb') as f:
    train_s = pickle.load(f)    
with open('noise.pkl','rb') as f:
    noise_s = pickle.load(f)    
with open('signal.pkl','rb') as f:
    signal_s = pickle.load(f)    
with open('X_list.pkl','rb') as f:
    complex_s = pickle.load(f)

All the signals in the dataset are of different lengths. Therefore, I am capturing the legths of the signals in a list which will be used later.

In [4]:
lens = []
for i in range(1200):
    lens.append(len(train_s[i]))

IBM targets are created as below by doing an Elementwise comparison and multiplying the TRUE/ FALSE values with 1 as TRUE x 1 = 1 and FALSE x 1 = 0

In [5]:
IBM_Targets = []
for i in range(1200):
    IBM_Targets.append(np.greater(signal_s[i],noise_s[i])*1)

## Creating the Computational Graph ##

I have made use of a 2 layer GRU network by using thr MultiRNNCell utility of Tensorflow explained by https://danijar.com/introduction-to-recurrent-networks-in-tensorflow/. I am keeping the learning rate to be 0.001 using the AdamOptimizer. The graph has been defined in such a way that it can take variable length signals as inputs. The number of units per layer is 513. As mentioned the dropout has been kept really low to regularize the network.

A fully connected layer has been used at the outputs which requires us to flatten the logits and the Y targets for comparison.
MSE(Mean Squared Error) has been used to compute the loss as I felt that it is appropriate to do so from a Weiner Filter perspective where it makes use of the MSE to reduce the error.

In [44]:
from tensorflow.contrib import rnn
tf.reset_default_graph()

num_layers = 2
num_units = 513

X = tf.placeholder("float", [None,None, 513])
Y = tf.placeholder("float", [None,None, 513])
keep_prob = tf.placeholder(tf.float32)

lstm_cells = []
for i in range(num_layers):
    lstm_cell = rnn.LSTMCell(num_units, forget_bias=1.0)
    lstm_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=keep_prob)
    lstm_cells.append(lstm_cell)
        
    lstm_cell = rnn.MultiRNNCell(lstm_cells)

output, states = tf.nn.dynamic_rnn(lstm_cell, X, dtype=tf.float32)

logits = tf.contrib.layers.fully_connected(output, 513, activation_fn=None)
prediction = logits

flat_Y = tf.reshape(Y, [-1] + Y.shape.as_list()[2:])
flat_logit = tf.reshape(logits, [-1] + logits.shape.as_list()[2:])

loss_op =tf.losses.mean_squared_error(predictions=flat_logit, labels=flat_Y)
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss_op)

correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

This is where I am training the network and saving the model by giving it a batch of 10 signals at a time. The saver function has been used so that I can reconstruct and check the signals later for the SNR.

In [45]:
with tf.Session() as sess: 

    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    for epoch in range(22):
        loss = 0
        for i in range(0, 1200,10):
            batch_target = np.array(IBM_Targets[i:i+10])
            batch_data = np.array(train_s[i:i+10])
            c, opt = sess.run([loss_op, optimizer], {X: batch_data, Y: batch_target, keep_prob: 0.9})
            
            loss = loss + c
        print('Loss at Epoch',epoch+1, 'is: ', loss)
    saver.save(sess, './LSTM-Shreyas')
    
    pr = []
    for i in range(1200):
        pr.append(sess.run(prediction, feed_dict={X: np.reshape(train_[i],(-1, lens[i], 513)), Y: np.reshape(targ_[i],(-1, lens[i], 513)), keep_prob: 0.9}))
    
    pr_test = []; z=[]
    for i in range(400):
        z.append(np.reshape(complex_test[i],(-1, lens_test[i], 513)))
    for i in range(400):
        pr_test.append(sess.run(prediction, feed_dict={X: z[i], keep_prob: 0.9}))        

Loss at Epoch 1 is:  27.096813574433327
Loss at Epoch 2 is:  23.691804632544518
Loss at Epoch 3 is:  21.674143686890602
Loss at Epoch 4 is:  20.193457528948784
Loss at Epoch 5 is:  19.349003985524178
Loss at Epoch 6 is:  18.708738461136818
Loss at Epoch 7 is:  18.191173873841763
Loss at Epoch 8 is:  17.649338118731976
Loss at Epoch 9 is:  17.247229605913162
Loss at Epoch 10 is:  16.912410587072372
Loss at Epoch 11 is:  16.661627054214478
Loss at Epoch 12 is:  16.401736311614513
Loss at Epoch 13 is:  16.11760475486517
Loss at Epoch 14 is:  15.9103574603796
Loss at Epoch 15 is:  15.66160311549902
Loss at Epoch 16 is:  15.404596500098705
Loss at Epoch 17 is:  15.280180156230927
Loss at Epoch 18 is:  15.086727887392044
Loss at Epoch 19 is:  14.913833811879158
Loss at Epoch 20 is:  14.856808871030807
Loss at Epoch 21 is:  14.768780991435051
Loss at Epoch 22 is:  14.516958005726337


/opt/Anaconda3-5.0.1/envs/e533/lib/python3.6/site-packages/numpy/core/numeric.py:492: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


Taking one signal from the network an reconstucting it to check if the noise is being filtered on the Training data. 

In [46]:
with tf.Session() as sess:
    saver.restore(sess, "./LSTM-Shreyas")
    pr_1 = sess.run(prediction, {X: np.reshape(train_s[1],(-1,65,513)), Y: np.reshape(IBM_Targets[1],(-1,65,513)), keep_prob: 0.9})
    recons1_matrix = np.transpose(np.reshape(np.multiply(pr_1, complex_s[1]), (65,513)))
    

INFO:tensorflow:Restoring parameters from ./LSTM-Shreyas


The reconstructed signal has been filtered really well as compared to the actual signal:  '/opt/e533/timit-homework/tr/trx0001.wav'

In [47]:
temp = librosa.istft(recons1_matrix, hop_length = 512)
librosa.output.write_wav('recons_op.wav', temp, sr)

import IPython.display as ipd
ipd.Audio('./recons_op.wav')

The actual NOISY signal

In [48]:
import IPython.display as ipd
ipd.Audio('/opt/e533/timit-homework/tr/trx0001.wav')

If you try listening to the above two signals, we can see that the model is training decently

The following pickle files have been used to load the validation data faster.

In [49]:
import pickle
with open('train_v.pkl','rb') as f:
    train_ = pickle.load(f)    
with open('noise_v.pkl','rb') as f:
    noise_ = pickle.load(f)    
with open('signal_v.pkl','rb') as f:
    signal_ = pickle.load(f)    
with open('targets_v_ibm.pkl','rb') as f:
    targ_ = pickle.load(f)

# Validation Step

Reconstructing all the validation signals

In [50]:
recons_matrix_v = []
for i in range(1200):
    recons_matrix_v.append(np.transpose(np.reshape(np.multiply(pr[i], complex_s[i]), (-1,lens[i],513))))

In [51]:
temp_v_istft = []; S_signal = []
for i in range(1200):
    temp_v_istft.append(librosa.istft(recons_matrix_v[i], hop_length = 512))
    S_signal.append(librosa.istft((signal_[i]), hop_length=512))

Calculating the SNR on the validation data.

In [52]:
snr = []
for i in range(1200):
    snr.append(10 * np.log10(sum(pow((S_signal[i]), 2))/ sum(pow((S_signal[i])-temp_v_istft[i], 2))))

In [53]:
print('mean: ', np.mean(snr))
print('min: ', np.min(snr))
print('max: ', np.max(snr))

mean:  2.3423377030584884
min:  -0.9922817990582469
max:  8.061181438739705


The following shows that 1186 out of 1200 signals have a positive SNR with the above statistics showing the mean, min and max. This is the best set of parameters that I could try out given the 2 layered architecture with 513 units each. 

I could get a 18% max in the previous work that I have done. But, the mean and min get affected, i.e. more signals get a negative SNR at the cost of overfitting to some signals. This according to me is not good, for overall performance. 

In [54]:
number_filtered = [i for i in snr if i>=0]

In [55]:
print('Number of Signals with Positive SNR:', len(number_filtered))


Number of Signals with Positive SNR: 1186


Loading the Test Signals Below:

In [21]:
test_data = []
complex_test = []

import os, re
for f in sorted(glob.iglob("/opt/e533/timit-homework/te/te*")):
        st, sr=librosa.load(f, sr=None)
        X=librosa.stft(st, n_fft=1024, hop_length=512)
        X_abs = np.abs(X)
        test_data.append(np.transpose(X_abs))
        complex_test.append(np.transpose(X))

In [22]:
lens_test = []
for i in range(400):
    lens_test.append(len(test_data[i]))

# Reconstructing the test signals

In [56]:
recons_matrix_test = []
for i in range(400):
    recons_matrix_test.append(np.transpose(np.reshape(np.multiply(pr_test[i], complex_test[i]), (-1,lens_test[i],513))))

In [59]:
for i in range(400):
    temp = librosa.istft(recons_matrix_test[i], hop_length = 512)
    librosa.output.write_wav(('./Recon_Test/shreyas_recons_test'+str(i)+'.wav'), temp,sr = 16000)

Here are the reconstructed signals stored in the folder: Recon_Test